In [2]:
from pandas import read_csv
from torch.optim import SGD
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from torch import randperm
from config import DATA_DIR
from src.data_set import RatingsDataset
from src.loss import MiningOutliersLoss
from src.model import MF
from src.runner import Runner
from src.utils import create_dataset, mine_outliers, DataConverter, DataProcessor

"""
The Deam dataset is based on Arousal-Valence 2D emotional model.
The Valence/Arousal ratings were collected using Amazon Mechanical Turks service.
Each turk from the collected crowd were asked to mark his own emotion for the current song on a 2D plane, Arousal/Valence.
For more information please read: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0173392
"""

DF_PATH = f"{DATA_DIR}" \
          f"/DEAM/annotations/annotations per each rater/" \
          f"song_level/static_annotations_songs_1_2000.csv"

def select_n_random(trainset: RatingsDataset):
    """
    Selects n random data points and their corresponding labels from a dataset
    """
    perm = randperm(len(trainset))
    return trainset[perm][:100]

In [ ]:
"""
This block of code calculates the outliers alongside the valence axis
"""
columns = ["workerID", "SongId", "Valence"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]

data_converter = DataConverter(
        original_df=original_df, n_random_users=10, n_ratings_per_random_user=200
    )
data_processor = DataProcessor(original_df=data_converter.original_df)

valence_model = MF(
    n_users=data_converter.n_users,
    n_items=data_converter.n_item,
)

criterion = MiningOutliersLoss(data_converter=data_converter, data_processor=data_processor)
optimizer = SGD(valence_model.parameters(), lr=5, weight_decay=1e-5)
runner = Runner(
    model=valence_model,
    criterion=criterion,
    optimizer=optimizer,
)

train_set = create_dataset(data_converter=data_converter)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)
users, items, ratings = select_n_random(train_set)

epochs = 50
with SummaryWriter("runs/DEAM_experiment_1") as writer:
    writer.add_graph(valence_model, (users, items))

    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
        print(f"epoch={epoch + 1}, loss={epoch_loss}")

100%|██████████| 20/20 [00:31<00:00,  1.60s/batch, train_loss=0.467]


epoch=1, loss=11.493567992374818


100%|██████████| 20/20 [00:24<00:00,  1.20s/batch, train_loss=0.364]


epoch=2, loss=7.903910919715619


100%|██████████| 20/20 [00:27<00:00,  1.37s/batch, train_loss=0.306]


...

100%|██████████| 20/20 [00:25<00:00,  1.30s/batch, train_loss=0.0395]


epoch=20, loss=1.0134161180956611


100%|██████████| 20/20 [00:19<00:00,  1.01batch/s, train_loss=0.0368]


epoch=21, loss=0.9291339572709183


 10%|█         | 2/20 [00:03<00:26,  1.48s/batch, train_loss=0.0372]

In [3]:
"""
This block of code calculates the outliers alongside the arousal axis
"""
columns = ["workerID", "SongId", "Arousal"]
original_df = read_csv(DF_PATH, skipinitialspace=True, usecols=columns)
original_df.columns = ["user_id", "item_id", "rating"]

data_converter = DataConverter(
        original_df=original_df, n_random_users=10, n_ratings_per_random_user=200
    )
data_processor = DataProcessor(original_df=data_converter.original_df)

arousal_model = MF(
    n_users=data_converter.n_users,
    n_items=data_converter.n_item,
)

criterion = MiningOutliersLoss(data_converter=data_converter, data_processor=data_processor)
optimizer = SGD(arousal_model.parameters(), lr=5, weight_decay=1e-5)
runner = Runner(
    model=valence_model,
    criterion=criterion,
    optimizer=optimizer,
)

train_set = create_dataset(data_converter=data_converter)
train_load = DataLoader(train_set, batch_size=1000, shuffle=True)
users, items, ratings = select_n_random(train_set)

epochs = 50
with SummaryWriter("runs/DEAM_experiment_1") as writer:
    writer.add_graph(arousal_model, (users, items))

    for epoch in range(epochs):
        epoch_loss = runner.train(train_loader=train_load, epoch=epoch, writer=writer)
        print(f"epoch={epoch + 1}, loss={epoch_loss}")

100%|██████████| 18/18 [00:07<00:00,  2.48batch/s, train_loss=0.215]


epoch=1, loss=4.8530552320072164


100%|██████████| 18/18 [00:07<00:00,  2.42batch/s, train_loss=0.184]


epoch=2, loss=3.582961639641788


100%|██████████| 18/18 [00:07<00:00,  2.39batch/s, train_loss=0.156]


...

epoch=48, loss=0.15975271047042963


100%|██████████| 18/18 [00:07<00:00,  2.40batch/s, train_loss=0.0101] 


epoch=49, loss=0.16075622285108157


100%|██████████| 18/18 [00:07<00:00,  2.37batch/s, train_loss=0.00942]

epoch=50, loss=0.16043610548509238


In [4]:
valence_outliers = mine_outliers(model=valence_model, data_converter=data_converter)
arousal_outliers = mine_outliers(model=arousal_model, data_converter=data_converter)

items_group_by_users = data_converter.original_df.groupby("user_id")
combined_outliers = {}
for user_id, valence_dist in valence_outliers.items():
    arousal_dist = arousal_outliers[user_id]
    combined_outliers[user_id] = valence_dist + arousal_dist

combined_outliers = dict(sorted(combined_outliers.items(), key=lambda item: item[1]))
for user_id, item_id in combined_outliers.items():
    number_of_items = len(items_group_by_users.get_group(user_id))
    print(f"user: {user_id}, dist: {item_id}, #items: {number_of_items}")

user: d88c800327bffffea5562e23c276ede3, dist: -8.16646671295166, #items: 2
user: b37092cf23b42f8b8497d8ba89be157a, dist: -7.9408345222473145, #items: 2
user: 65794ea9f5122952403585a237bc5e52, dist: -6.504777908325195, #items: 3
user: 49be5653eaba26f1eb60fd9d63f23502, dist: -3.5293586254119873, #items: 3
user: 651938620e6e6c78bfa7854784fe62c2, dist: -2.5150487422943115, #items: 3
user: 15ec33e862185406170ff931583b014f, dist: -2.2195191383361816, #items: 4
user: d0c51e42ea093dc9a9a98ef888637c8e, dist: -2.0096826553344727, #items: 2
user: da37d1548ffd0631809f7be341e4fe4d, dist: -1.3743306398391724, #items: 3
user: 38531641e6c0628757776b0088bcc854, dist: -0.7693064212799072, #items: 7
user: 807f0025a626896f04566aa37cfbce0d, dist: -0.43722257018089294, #items: 3
user: a186cdd58a92051b7c73adc9bd6e65ca, dist: -0.29314088821411133, #items: 7
user: 27f51a4a7fe8565d26cadb88584441e5, dist: -0.279450386762619, #items: 2
user: 5b044cf509da1d8444b6f60c465240ef, dist: -0.1574951410293579, #items: 3
u